In [1]:
class Literal():
    """
    Class to represent a logical literal
    Attributes (in corresponding order):
    - Name of the literal
    - Sign of the literal, which is True if the literal is positive
    and is False if the literal is negative, defaulted to True
    """
    def __init__(self, name='', sign=True):
        """Initialize the literal"""
        self._name = name
        self.sign = sign
        
    def __neg__(self):
        """Define a negated literal"""
        return Literal(self._name, not self.sign)
    
    def __eq__(self, other):
        """Set that two literals are equal regardless of their sign"""
        return self._name == other._name
    
    def __hash__(self):
        """Define hash function to hash the name attribute"""
        return hash(self._name)
    
    def __repr__(self):
        return self.__str__()

    def __str__(self):
        if self.sign:
            return self._name
        else:
            return '-'+self._name

In [2]:
import copy
def getSymbol(KB):
    """Parse the KB to get all the literals used"""
    symbol_list = set()
    for clause in KB:
        for symbol in clause:
            if symbol not in symbol_list:
                if symbol.sign:
                    symbol_list.add(symbol)
                else:
                    # If the literal is negative,
                    # store the positive literal instead
                    symbol_list.add(-symbol)
    return symbol_list

def degreeHeuristic(clauses, symbols, model):
    """Implementation of the degree heuristic"""
    degree = {}
    # Find all symbols that are yet to be assigned
    # and store them in the dict with degree set to 0
    for sym in symbols:
        if not sym in model:
            degree[sym] = 0
    
    # If there is no symbols that are yet to be assigned,
    # return None
    if len(degree) == 0:
        return None
    
    # If there is only one symbol that is yet to be assigned,
    # return the symbol
    if len(degree) == 1:
        return max(degree, key=degree.get)
    
    # Calculate the degree for all unassigned symbols
    for clause in clauses:
        for symbol in clause:
            if symbol in degree:
                degree[symbol] += 1
    # Return the symbol with the highest degree
    return max(degree, key=degree.get)

def pureSymbol(clauses, model):
    """Implementation of the pure symbol heuristic"""
    positive_literals = set()
    negative_literals = set()
    # Find all the positive literals and negative literals
    for clause in clauses:
        for symbol in clause:
            if symbol.sign and symbol not in positive_literals:
                positive_literals.add(symbol)
            if not symbol.sign and symbol not in negative_literals:
                negative_literals.add(-symbol)
    # Perform xor on the two sets to get the set of pure symbols
    pure_symbols = positive_literals ^ negative_literals
    
    # Use the degree heuristic to find the pure symbol that is yet 
    # to be assigned and occurs in the most number of clauses 
    max_degree = degreeHeuristic(clauses, pure_symbols, model)
    # If the symbol is a postive literal, then we need to assign
    # True to make the clause True
    if max_degree in positive_literals:
        return max_degree, 'true'
    # If the symbol is a negative literal, then we need to assign
    # False to make the clause True
    # This also takes into account the case where the symbol is null.
    else:
        return max_degree, 'false'

def unitClause(truth_clauses, model):
    """Implementation of the unit clause heuristic"""
    # truth_clauses is a dict of the clauses in the KB and the
    # truth value of their literals in the given model
    symbols = set()
    for clause in truth_clauses:
        # If the clause is not True in the given model
        if 'true' not in truth_clauses[clause]:
            # Count many literals are still unassigned
            count = 0
            for value in truth_clauses[clause]:
                if value == 'unknown':
                    count += 1
            # If there is only one literal without an assigned
            # truth value, the clause is a unit clause.
            # Store the symbol to be assigned.
            if count == 1:
                for symbol in clause:
                    if symbol not in symbols:
                        if symbol.sign:
                            symbols.add(symbol)   
                        else:
                            symbols.add(-symbol) 
    # Use the degree heuristic to find the unit clause with 
    # the symbol that is yet to be assigned and occurs in
    # the most number of clauses
    max_degree = degreeHeuristic(truth_clauses, symbols, model)
    # If the symbol is null, return None
    if max_degree is None:
        return None, 'false'
    # If the symbol is a postive literal, then we need to assign
    # True to make the clause True
    if max_degree.sign:
        return max_degree, 'true'
    # If the symbol is a negative literal, then we need to assign
    # False to make the clause True
    else:
        return max_degree, 'false'

def DPLLSatisfiable(KB):
    # Get the symbols used in the KB
    symbols = getSymbol(KB)
    # Turn the clauses into frozen sets to hash them later
    clauses = [frozenset(clause) for clause in KB]
    return DPLL(clauses, symbols, {})

def DPLL(clauses, symbols, model):
    # Store the truth value of each clause in the given model
    truth_values = []
    # The dict of the clauses in the KB and the truth value of
    # their literals in the given model
    truth_clauses = {}
    # Evaluate truth value of KB in the given model
    for clause in clauses:
        clause_truth_value = set({})
        for symbol in clause:
            # If the symbol has been assigned in the model
            if symbol in model:
                if symbol.sign:
                    clause_truth_value.add(model[symbol])
                    # If the literal is positive and it is True
                    # then the whole clause is True, so break
                    # out of the for loop
                    if model[symbol] == 'true':
                        break
                elif model[symbol] == 'true':
                    clause_truth_value.add('false')
                else:
                    # If the literal is negative and it is False
                    # then the whole clause is True, so break
                    # out of the for loop
                    clause_truth_value.add('true')
                    break
            # If not assigned yet, mark it as unknown
            else:
                clause_truth_value.add('unknown')   
        truth_clauses[clause] = clause_truth_value
        # Assigning truth value to the whole clause
        # If the clause has a True in it, then it is True
        if 'true' in clause_truth_value:
            truth_values.append(True)
        # If the clause does not have a True in it, and it still
        # has unknown literals, then its truth value is unknown
        # I store None here because it evaluates to False, but it
        # is not the same value as False
        elif 'unknown' in clause_truth_value:
            truth_values.append(None)
        # If the clause's literals all evaluate to False, then the
        # clause is False and the whole KB is False, so KB is not
        # satisfiable in the given model
        else:
            return False, {}
    # If all clauses are True
    if all(truth_values):
        # Mark the unassigned symbols as 'free'
        for sym in symbols:
            if sym not in model:
                model[sym] = 'free'
        return True, model
    # At this point, the KB does not have a False clause (else it 
    # would have terminated early), and not all of its clauses are 
    # True so we need to continue assigning values to symbols.
    else:
        # Get next symbol to assign using pure symbol heuristic
        sym, value =  pureSymbol(clauses, model)
        if sym is not None:
            new_symbols = copy.deepcopy(symbols)
            new_symbols.remove(sym)
            satisfiability, m = DPLL(clauses, new_symbols, \
                                     {**model, sym: value})
            if satisfiability:
                return satisfiability, m
        
        # If there is no pure symbol unassigned, then use unit
        # clause heuristic
        sym, value =  unitClause(truth_clauses, model)
        if sym is not None:
            new_symbols = copy.deepcopy(symbols)
            new_symbols.remove(sym)
            satisfiability, m = DPLL(clauses, new_symbols, \
                                     {**model, sym: value})
            if satisfiability:
                return satisfiability, m
        # If there is no unit clause, then use degree heuristic
        # to choose the next symbol
        sym = degreeHeuristic(clauses, symbols, model)
        new_symbols = copy.deepcopy(symbols)
        new_symbols.remove(sym)
        satisfiability, m = DPLL(clauses, new_symbols, \
                                 {**model, sym: 'true'})
        if satisfiability:
            return satisfiability, m
        else:
            return DPLL(clauses, new_symbols,{**model, sym:'false'})

In [3]:
A = Literal('A')

B = Literal('B')

C = Literal('C')

D = Literal('D')

KB = [{A,B},{A,-C},{-A,B,D}]
KB1 = [{-A, B}, {A, -C}, {-B}, {C}]

In [4]:
DPLLSatisfiable(KB)

(True, {A: 'free', B: 'true', C: 'false', D: 'true'})

In [5]:
DPLLSatisfiable(KB1)

(False, {})

In [6]:
E = Literal('E')

F = Literal('F')

KB2 = [{-A, B, E}, {-B, A}, {-E, A}, {-E, D}, {-C, -F, -B}, {-E, B}, {-B, F}, {-B, C}]

In [7]:
DPLLSatisfiable(KB2)

(True, {A: 'false', B: 'false', C: 'free', D: 'true', E: 'false', F: 'free'})